# Notebook summary of work 

In [3]:
parent_folder_path = "/Users/haleyoro/Desktop/" # work on library computer
# parent_folder_path = "/Users/Haley/Desktop/Neuroscience Lab/Repository/" # work on local computer

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pyabf

Matplotlib is building the font cache; this may take a moment.


Looking at the waveforms (from `Plotting` folder)

Annotations sheet --`App Script`--> unformatted csv files 

unformatted csv files --`process_excel`--> formatted csv files 

## `annotations_loader` : Make `sheets` dict with 38 dataframes of all processed .csvs

Load the cell types data frame to include info when reading .csv sheets

In [14]:
cell_types_df = pd.read_csv(parent_folder_path+"murray-neuroscience-lab/Excel processor/List of cells.csv")
cell_types_df = cell_types_df.dropna(how='all')
cell_types_df = cell_types_df.iloc[:,:4]
cell_types_df.reset_index(drop=True,inplace=True)
sheets_types = cell_types_df["Cell"].unique().tolist()
print("Number of labeled sheets:",len(sheets_types))

Number of labeled sheets: 29


Create a dictionary looping through sheet names to load each sheet annotation as a data frame 
(.csv files were re-formatted using functions in `process_excel`, looping through the downloaded excel sheets from the annotations sheet in google drive. excel sheets were downloaded as .csvs using App Script code)

In [9]:
sheet_names_df = pd.read_csv(parent_folder_path+'murray-neuroscience-lab/Processed CSVs/sheet_names.csv', header=None)
sheet_names = sheet_names_df.iloc[:,0].values
sheets = {}

for sheet in sheet_names:
    file_path = parent_folder_path+"murray-neuroscience-lab/Processed CSVs/"+sheet+".csv"
    df = pd.read_csv(file_path)
    df[["Trace name","Tags","Type"]] = df[["Trace name","Tags","Type"]].astype("string")
    if sheet in sheets_types:
        types = cell_types_df[cell_types_df["Cell"]==sheet]
        df["Cell Type"] = types.iloc[0][1]
        df["Input Resistance"] = types.iloc[0][2]
        df["Motoneuron"] = types.iloc[0][3]
    sheets[sheet] = df

sheets

/var/folders/40/qf3btjqs32l1pmzzyts7w9pmh2xrdg/T/ipykernel_66755/4001062656.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["Cell Type"] = types.iloc[0][1]
/var/folders/40/qf3btjqs32l1pmzzyts7w9pmh2xrdg/T/ipykernel_66755/4001062656.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["Input Resistance"] = types.iloc[0][2]
/var/folders/40/qf3btjqs32l1pmzzyts7w9pmh2xrdg/T/ipykernel_66755/4001062656.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by posi

{'2012_04_25_cell1':           Trace name  ID  On time     Freq               Tags        Type  \
 0    2012_04_25_0004   1  1285.99      NaN  bout start; shock  Inhibitory   
 1    2012_04_25_0004   2  1305.92  50.1756               <NA>  Inhibitory   
 2    2012_04_25_0004   3  1340.95  28.5470               <NA>  Inhibitory   
 3    2012_04_25_0004   4  1371.30  32.9489               <NA>  Inhibitory   
 4    2012_04_25_0004   5  1405.23  29.4724           bout end  Inhibitory   
 ..               ...  ..      ...      ...                ...         ...   
 153  2012_04_25_0002  13  4726.57  39.7141               <NA>  Excitatory   
 154  2012_04_25_0002  14  4755.30  34.8068               <NA>  Excitatory   
 155  2012_04_25_0002  15  4780.80  39.2157               <NA>  Excitatory   
 156  2012_04_25_0002  16  4808.89  35.5999               <NA>  Excitatory   
 157  2012_04_25_0002  17  4846.59  26.5252           bout end  Excitatory   
 
      Seconds Cell Type  Input Resistance 

Keys are names of sheets

In [15]:
print("Number of sheets:", len(sheets.keys()))
print("Number of labeled sheets:",len(sheets_types))
sheets.keys()

Number of sheets: 39
Number of labeled sheets: 29


dict_keys(['2012_04_25_cell1', '2012_04_25_cell3', '2012_04_27_cell1', '2012_06_22_cell3', '2012_06_25_cell3', '2012_06_29', '2012_08_01_cell1', '2012_08_01_cell3', '2012_08_07', '2012_08_31_cell1', '2012_08_31_cell2', '2012_08_31_cell3', '2012_08_31_cell4', '2012_10_04_cell2', '2012_12_03_cell1', '2012_12_04_cell1', '2012_12_04_cell2', '2012_12_05_cell1', '2012_12_05_cell3', '2012_12_05_cell4', '2012_12_06_cell1', '2012_12_06_cell2', '2012_12_06_cell3', '2012_12_06_cell4', '2012_12_06_cell5', '2012_12_06_cell6', '2013_03_20_cell1', '2013_03_21_cell1', '2013_03_21_cell2', '2013_03_21_cell3', '2013_03_21_cell4', '2013_03_21_cell5', '2013_03_22_cell1', '2013_03_22_cell2', '2013_03_22_cell3', '2013_03_22_cell4', '2013_03_22_cell5', '2013_03_22_cell6', '2012_10_04_cell1'])

Using one sheet name to access the formatted dataframe of the annotations, includes cell type and resistance

In [12]:
sheets['2012_04_25_cell1']

,Trace name,ID,On time,Freq,Tags,Type,Seconds,Cell Type,Input Resistance,Motoneuron
0,2012_04_25_0004,1,1285.99,NaN,bout start; shock,Inhibitory,1.28599,vRoP,74.45,primary
1,2012_04_25_0004,2,1305.92,50.1756,<NA>,Inhibitory,1.30592,vRoP,74.45,primary
2,2012_04_25_0004,3,1340.95,28.5470,<NA>,Inhibitory,1.34095,vRoP,74.45,primary
3,2012_04_25_0004,4,1371.30,32.9489,<NA>,Inhibitory,1.37130,vRoP,74.45,primary
4,2012_04_25_0004,5,1405.23,29.4724,bout end,Inhibitory,1.40523,vRoP,74.45,primary
...,...,...,...,...,...,...,...,...,...,...
153,2012_04_25_0002,13,4726.57,39.7141,<NA>,Excitatory,4.72657,vRoP,74.45,primary
154,2012_04_25_0002,14,4755.30,34.8068,<NA>,Excitatory,4.75530,vRoP,74.45,primary
155,2012_04_25_0002,15,4780.80,39.2157,<NA>,Excitatory,4.78080,vRoP,74.45,primary
156,2012_04_25_0002,16,4808.89,35.5999,<NA>,Excitatory,4.80889,vRoP,74.45,primary


## `binwaves_` : make abfs dict to store .abf files for each of the 38 sheets

make `abfs` dict: contains the abf file of each trace in the annotation df sheet

In [10]:
def make_waveforms(abf,df):
    '''
    Function that takes an abf file and a df of the annotations
    Returns a dictionary with waveforms labeled by their frequency
    '''
    abf.setSweep(sweepNumber=0, channel=0)
    x = abf.sweepX
    y = abf.sweepY
    abf_df = pd.DataFrame({
        'Time': x,
        'Current': y
    })
    # df["On time"] = pd.to_numeric(df["On time"], errors="coerce")
    # df["Seconds"] = df['On time']*0.001
    waveforms = {}
    for i in range(len(df) - 1):
        t_0 = df.iloc[i,6] 
        t_f = df.iloc[i+1,6]
        phase_0 = int(t_0 * 100000)
        phase_1 = int(t_f * 100000)
        abf_waveform = abf_df.loc[phase_0:phase_1]
        phase = (abf_waveform["Time"] - t_0) / (t_f - t_0)
        abf_waveform["Phase"] = phase
        
        # NORMALIZE CURRENT ALSO, test this:
        y_max = max(abf_waveform["Current"])
        y_min = min(abf_waveform["Current"])
        normalized = (abf_waveform["Current"] - y_min) / (y_max - y_min)
        abf_waveform["Normalized Current"] = normalized
        #=====

        # Dict keys
        freq = 1 / (t_f-t_0) # CHECK = df.loc[i + 1, "Freq"]
        cell_type = df.iloc[0,5]
        signal_type = df.iloc[0,7]
        fast_slow = df.iloc[0,8:9] # 8:resistance 9:prim/sec low/high rin need to see what is fast/slow
        key = (freq,cell_type,signal_type,fast_slow)
        waveforms[key] = abf_waveform

    return waveforms

need abfs[trace] and sheets[sheet[sheet["Trace name"]==Trace]]

use loops in annotations loader to make mega dict

TEST: add nested dicts with abf files into `sheets` 

In [ ]:
for sheet in sheets.keys():
    df = sheets[sheet]
    traces = df["Trace name"].unique().tolist() # creates an array of the trace names annotated in the sheet
    # load the abf files for each sheet into a dictionary 
    abfs = {}
    for trace in traces:
        abfs[trace] = []
        file_path2 = "/Users/Haley/Downloads/.abf files annotated/" + trace +".abf"
        abf = pyabf.ABF(file_path2)
        abfs[trace]= abf
    # adds the dictionary to sheets 
    sheets[sheet] = {
        "annotations": sheets[sheet],
        "abfs": abfs
    }

TEST: loop to use `sheets` to make a dict `all_waveforms`

In [ ]:
all_waveforms = {}
for sheet in sheets.keys():
    traces_df_mkwf = sheets[sheet]["annotations"]
    abfs_mkwv = sheets[sheet]["abfs"]
    traces = traces_df_mkwf["Trace name"].unique().tolist()
    for trace in traces:
        abf_mkwv = abfs_mkwv[trace]
        df_mkwv = traces_df_mkwf[traces_df_mkwf["Trace name"]==trace]
        waveforms = make_waveforms(abf_mkwv,df_mkwv)

        # NEED TO FIGURE OUT HOW TO JOIN WAVEFORMS W ALL WAVEFORMS DICTIONARIES HERE

In [ ]:
def bin_wave(onewave):
    # Make edges for the bin
    bins = np.linspace(0, 1, 51, endpoint = True)

    # Cut the phase into bins
    onewave['Phase Bin'] = pd.cut(onewave['Phase'], bins=bins, include_lowest=True)
    # print(onewave)

    # Group by the bins and calculate the average current
    # binned_avg = onewave.groupby(by='Phase Bin','Current')
    binned_avg = onewave.groupby('Phase Bin')['Current'].mean().reset_index()
    # print(binned_avg)

    # Get phase column for plotting (middle of bin)
    phase = np.arange(0.01, 1.00001, 0.02)
    # len(phase)
    binned_avg['Phase'] = phase
    return binned_avg

bin `all_waveforms` and store in `binned_waveforms`

In [ ]:
binned_waveforms = {}
for wave in all_waveforms.keys():
    binned = bin_wave(all_waveforms[wave])
    binned_waveforms[wave] = binned

NEXT: access keys to average across the key types and plot
- current type: excitatory, inhibitory, cell-attached
- frequency bins:
- rin: low/mid/high or fast/slow
- cell type: CaP,vRoP,MiP,dRoP,dVS,dSMN,vSMN,iSMN

function(current,freq,rin,type) -> plot waveform